In [1]:
import pandas as pd
import numpy as np
import wfdb
import ast

In [2]:
%run target_dict.ipynb

In [3]:
# путь к файлу и кол-во семплов
path = 'ptb-xl/1.0.1/'
sampling_rate=100

In [4]:
#для формирования X по ЭКГ (кол-во ЭКГ(не пациентов!), 100 записей в сек, кол-во электродов)
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    
    return data

In [5]:
# для data_X
def load_raw_data_X(df, sampling_rate, path):
    data_X = []
    if sampling_rate == 100:
        for f in df.filename_lr:
            data_X.append([df.filename_lr, wfdb.rdsamp(path+f)])
    else:
        for f in df.filename_hr:
            data_X.append([df.filename_hr, wfdb.rdsamp(path+f)])        
    
    return data_X

In [14]:
# проба
path = 'ptb-xl/1.0.1/'
f = 'records100/00000/00001_lr'
d = wfdb.rdsamp(path+f)
d[1]

{'fs': 100,
 'sig_len': 1000,
 'n_sig': 12,
 'base_date': None,
 'base_time': None,
 'units': ['mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV'],
 'sig_name': ['I',
  'II',
  'III',
  'AVR',
  'AVL',
  'AVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6'],
 'comments': []}

In [18]:
# загрузка даты с лейблами
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [9]:
# для обозначения суперкласса нормальный/не нормальный (в задаче другие таргеты)
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        
        if key in agg_df.index:
            diag = agg_df.loc[key].diagnostic_class
            if diag == 'NORM':
                tmp.append(diag)
           
            else:
                tmp.append('pathology')
                
    return list(set(tmp))

In [10]:
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [17]:
Y['patient_id'][1]

15709.0

In [11]:
# нам нужны свои таргеты согласно заданию - посмотрим, где их искать и обозначения
for i in Y.scp_codes:
    print(i)

{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'SBRAD': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IMI': 35.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SARRH': 0.0}
{'NORM': 80.0, 'SBRAD': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SARRH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'AFLT': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0}
{'AFLT': 100.0}
{'NORM': 100.0, 'SR': 0.0}
{'AFLT': 100.0, 'ABQRS': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'AFLT': 100.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NST_': 100.0, 'DIG': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'NST_': 100.0, 'DIG': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LPFB': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'AFLT': 100.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 

{'ISCIL': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'ALMI': 100.0}
{'IMI': 80.0, 'ASMI': 80.0, 'SR': 0.0}
{'IRBBB': 100.0, 'SR': 0.0}
{'ISCAL': 100.0, 'LVH': 50.0, 'SR': 0.0}
{'ASMI': 50.0, 'CRBBB': 100.0, 'LAFB': 100.0, 'ISCAL': 50.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'LVH': 100.0, 'ISC_': 100.0, 'PVC': 100.0, 'AFIB': 0.0}
{'ASMI': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 50.0, 'LVH': 100.0, 'ISC_': 100.0, 'LAFB': 100.0, 'PVC': 100.0}
{'LAFB': 100.0, 'IRBBB': 100.0, 'SR': 0.0}
{'ISCIL': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 15.0, 'LAO/LAE': 50.0}
{'NORM': 80.0, 'SR': 0.0}
{'ISCIL': 100.0, 'SR': 0.0}
{'IRBBB': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
{'LVH': 100.0, 'SR': 0.0}
{'ASMI': 50.0, 'CRBBB': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'ISC_': 100.0}
{'IMI': 100.0, 'NDT': 100.0, 'PVC': 100.0, 'SR': 0.0}
{'NORM': 100.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 

{'NORM': 100.0, 'SARRH': 0.0}
{'CRBBB': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'PVC': 100.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SBRAD': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'IVCD': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'STACH': 0.0}
{'NORM': 100.0, 'IRBBB': 100.0, 'SARRH': 0.0}
{'ALMI': 100.0, 'ASMI': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0}
{'NORM': 100.0, 'SBRAD': 0.0}
{'ALMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'AMI': 80.0, 'LVH': 100.0, 'DIG': 100.0, 'VCLVH': 0.0, 'QWAVE': 0.0, 'LOWT': 0.0, 'SR': 0.0}
{'NST_': 100.0, 'STD_': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'STACH': 0.0}
{'ASMI': 50.0, 'LMI': 15.0, 'VCLVH': 0.0, 'AFIB': 0.0}
{'LVH': 15.0, 'SR': 0.0}
{'NORM': 100.0, 'PVC': 0.0, 'STACH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 15.0, 'IMI': 15.0, 'LAFB': 100.0, 'NST_': 0.0, 'SR': 0.0}
{'ASMI': 100.0, 'ALMI': 100.0, 'IVCD': 100.0,

{'ASMI': 15.0, 'CRBBB': 100.0, 'AFIB': 0.0}
{'LVH': 35.0, 'PAC': 0.0, 'VCLVH': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'LOWT': 0.0, 'AFIB': 0.0}
{'IMI': 80.0, 'QWAVE': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'STACH': 0.0}
{'NDT': 100.0, 'PAC': 0.0, 'NST_': 0.0, 'STACH': 0.0}
{'LVH': 50.0, 'NDT': 100.0, 'STD_': 0.0, 'SR': 0.0}
{'NST_': 100.0, 'IVCD': 100.0, 'STD_': 0.0, 'SR': 0.0}
{'LVH': 50.0, 'VCLVH': 0.0, 'SR': 0.0}
{'IMI': 80.0, 'QWAVE': 0.0, 'AFIB': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'IMI': 15.0, 'IRBBB': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IMI': 15.0, 'INJAS': 100.0, 'INVT': 0.0, 'LOWT': 0.0, 'SR': 0.0}
{'ASMI': 15.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'LAO/LAE': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LAFB': 100.0, 'PVC': 100.0, 'LOWT': 0.0, 'STD_': 0.0, 'SR': 0.0}
{'LMI': 50.0, 'SR': 0.0}
{'ASMI': 50.0, 'LVH': 50.0, 'ISC_': 100.0, 'PVC': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0}
{'CRBBB': 100

{'NORM': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'STD_': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'PVC': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ILMI': 100.0, 'IVCD': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 15.0, 'QWAVE': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'1AVB': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'AFIB': 0.0}
{'IMI': 15.0, 'PVC': 100.0, 'SARRH': 0.0}
{'LAFB': 100.0}
{'IMI': 100.0, 'NDT': 100.0, 'SR': 0.0}
{'LVH': 100.0, 'VCLVH': 0.0, 'SARRH': 0.0}
{'NORM': 100.0, 'SARRH': 0.0}
{'IMI': 50.0, 'ASMI': 50.0, 'ABQRS': 0.0, 'SR': 0.0}
{'IMI': 100.0, 'IVCD': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'STD_': 0.0, 'SARRH': 0.0}
{'IRBBB': 100.0, 'AFIB': 0.0}
{'INJAS': 100.0, 'INJAL': 100.0, 'LVH': 100.0, 'INVT': 0.0, 'STD_': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'NT_': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'NT_': 0.0, 'SR': 0.0}
{'IMI': 35.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'STD_': 0.0, '

...

{'IMI': 100.0, 'PVC': 100.0, 'ABQRS': 0.0, 'SARRH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 100.0, 'ISC_': 100.0, 'AFIB': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'STACH': 0.0}
{'NST_': 100.0, 'AFIB': 0.0, 'AFLT': 0.0}
{'LAFB': 100.0, 'LVH': 100.0, 'PVC': 100.0, 'AFIB': 0.0}
{'ASMI': 100.0, 'LMI': 50.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'IMI': 100.0, 'LVH': 100.0, 'ISC_': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'LVH': 100.0, 'STD_': 0.0, 'VCLVH': 0.0, 'SR': 0.0}
{'LVH': 100.0, 'ISC_': 100.0}
{'ASMI': 15.0, 'LAFB': 100.0, 'LAO/LAE': 50.0, 'SR': 0.0}
{'IMI': 100.0, 'INJAS': 100.0, 'ISCAL': 100.0, 'STD_': 0.0, 'SR': 0.0}
{'RAO/RAE': 100.0, 'STACH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ISC_': 100.0, 'LVH': 100.0, 'SR': 0.0}
{'IRBBB': 100.0, 'SARRH': 0.0}
{'NORM': 100.0, 'SARRH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 35.0, 'STD_': 0.0, 'VCLVH': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IPLMI': 100.0, 'CRBBB': 100.0, 

{'IMI': 15.0, 'NST_': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'LMI': 100.0, 'LPR': 0.0, 'QWAVE': 0.0, 'LOWT': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'IMI': 50.0, 'LVH': 100.0, 'ISC_': 100.0, 'LAO/LAE': 100.0, 'STD_': 0.0, 'SBRAD': 0.0}
{'ISCLA': 100.0, 'LOWT': 0.0, 'SR': 0.0}
{'CRBBB': 100.0, '2AVB': 100.0}
{'LAFB': 100.0, 'NST_': 0.0, 'SR': 0.0}
{'ASMI': 100.0, 'IMI': 15.0, 'LAFB': 100.0, 'PAC': 0.0, 'QWAVE': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IMI': 50.0}
{'CLBBB': 100.0, '1AVB': 100.0, 'SBRAD': 0.0}
{'NORM': 100.0, 'AFIB': 0.0}
{'NST_': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 100.0, 'SEHYP': 100.0, 'QWAVE': 0.0, 'SR': 0.0}
{'IMI': 50.0, 'NDT': 100.0, 'LOWT': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'HVOLT': 0.0, 'SR': 0.0}
{'LAFB': 80.0, 'PAC': 0.0, 'SR': 0.0}
{'IMI': 35.0, 'NDT': 100.0, 'STACH': 0.0}
{'NDT': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'INJAS': 100.0, 'INJAL': 100.0, 'LVH': 100.0, 'STD_': 0.0, 'INVT

{'ASMI': 15.0, 'LVH': 50.0, 'ABQRS': 0.0, 'VCLVH': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SBRAD': 0.0}
{'IMI': 15.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NST_': 100.0, 'AFIB': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'WPW': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'LAFB': 100.0, '1AVB': 100.0, 'SR': 0.0}
{'WPW': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 50.0, 'PVC': 100.0, 'VCLVH': 0.0}
{'NORM': 50.0, 'LVOLT': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'IRBBB': 80.0, 'SR': 0.0}
{'ASMI': 100.0, 'ISCLA': 100.0, 'IVCD': 100.0, 'LAO/LAE': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'IMI': 100.0, 'ASMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'SR': 0.0}
{'CRBBB': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'IVCD': 100.0, 'PVC': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'IPMI': 100.0, 'LVH': 50.0, 'ISC_': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'ALMI': 50.0, '

{'NORM': 100.0, 'SR': 0.0}
{'ILMI': 50.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'STACH': 0.0}
{'IMI': 100.0, 'ASMI': 100.0, 'PVC': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0}
{'ASMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 50.0, '1AVB': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, '1AVB': 100.0, 'SR': 0.0}
{'PACE': 100.0}
{'ASMI': 100.0, 'ALMI': 100.0, 'LAFB': 100.0, 'SR': 0.0}
{'LVH': 50.0, 'VCLVH': 0.0, 'SVARR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 100.0, 'ISC_': 100.0, 'ILBBB': 100.0, 'PVC': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ISCAS': 100.0, 'ISCAL': 100.0, 'SR': 0.0}
{'IMI': 100.0, '1AVB': 100.0, 'PVC': 100.0, 'BIGU': 100.0, 'STACH': 0.0}
{'PACE': 100.0}
{'NORM': 100.0, 'ABQRS': 0.0, 'SVARR': 0.0}
{'ASMI': 100.0, 'ILMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'NST_': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'LMI': 15.0, 'CLBBB': 100.0, '1AVB': 100.0, 'SARRH': 0.0}
{

{'ISCIL': 100.0, 'PVC': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'ASMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IMI': 100.0, 'ASMI': 100.0, 'ABQRS': 0.0, 'SBRAD': 0.0}
{'ISCAL': 100.0, 'ISCIL': 100.0, 'AFIB': 0.0}
{'ISCIN': 50.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'IRBBB': 100.0, 'LVH': 100.0, 'ISC_': 100.0, 'VCLVH': 0.0, 'AFIB': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'AFIB': 0.0}
{'NORM': 15.0, 'SR': 0.0}
{'NDT': 100.0, 'AFIB': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'ASMI': 50.0, 'ILBBB': 100.0, 'LAFB': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IVCD': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NST_': 100.0, 'AFIB': 0.0}
{'NDT': 100.0, 'PVC': 100.0}
{'NORM': 50.0, 'STACH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'LAO/LAE': 100.0, 'SR': 0.

In [12]:
agg_df[['diagnostic','SCP-ECG Statement Description', 'AHA code', 'aECG REFID', 'CDISC Code',
       'DICOM Code']]

,diagnostic,SCP-ECG Statement Description,AHA code,aECG REFID,CDISC Code,DICOM Code
NDT,1.0,non-diagnostic T abnormalities,NaN,NaN,NaN,NaN
NST_,1.0,non-specific ST changes,145.0,MDC_ECG_RHY_STHILOST,NaN,NaN
DIG,1.0,suggests digitalis-effect,205.0,NaN,NaN,NaN
LNGQT,1.0,long QT-interval,148.0,NaN,NaN,NaN
NORM,1.0,normal ECG,1.0,NaN,NaN,F-000B7
IMI,1.0,inferior myocardial infarction,161.0,NaN,NaN,NaN
ASMI,1.0,anteroseptal myocardial infarction,165.0,NaN,NaN,NaN
LVH,1.0,left ventricular hypertrophy,142.0,NaN,C71076,NaN
LAFB,1.0,left anterior fascicular block,101.0,MDC_ECG_BEAT_BLK_ANT_L_HEMI,C62267,D3-33140
ISC_,1.0,ischemic ST-T changes,226.0,NaN,NaN,NaN


In [126]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21837 entries, 1 to 21837
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   patient_id                    21837 non-null  float64
 1   age                           21748 non-null  float64
 2   sex                           21837 non-null  int64  
 3   height                        6983 non-null   float64
 4   weight                        9429 non-null   float64
 5   nurse                         20328 non-null  float64
 6   site                          21819 non-null  float64
 7   device                        21837 non-null  object 
 8   recording_date                21837 non-null  object 
 9   report                        21837 non-null  object 
 10  scp_codes                     21837 non-null  object 
 11  heart_axis                    13332 non-null  object 
 12  infarction_stadium1           5626 non-null   object 
 13  i

In [14]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, NDT to 2AVB
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   description                    44 non-null     object 
 1   diagnostic                     44 non-null     float64
 2   form                           4 non-null      float64
 3   rhythm                         0 non-null      float64
 4   diagnostic_class               44 non-null     object 
 5   diagnostic_subclass            44 non-null     object 
 6   Statement Category             44 non-null     object 
 7   SCP-ECG Statement Description  44 non-null     object 
 8   AHA code                       22 non-null     float64
 9   aECG REFID                     12 non-null     object 
 10  CDISC Code                     13 non-null     object 
 11  DICOM Code                     9 non-null      object 
dtypes: float64(4), object(8)
memory usage: 5.7+ KB


In [15]:
agg_df['diagnostic_subclass'].unique()

array(['STTC', 'NST_', 'NORM', 'IMI', 'AMI', 'LVH', 'LAFB/LPFB', 'ISC_',
       'IRBBB', '_AVB', 'IVCD', 'ISCA', 'CRBBB', 'CLBBB', 'LAO/LAE',
       'ISCI', 'LMI', 'RVH', 'RAO/RAE', 'WPW', 'ILBBB', 'SEHYP', 'PMI'],
      dtype=object)

In [16]:
# 'AFIB': 100.0 - аритмия
# 'SBRAS' - брадикардия?
# по остальным вообще информации нет - лейблы (а в agg описание тоже не подходит, в основном про миокард и инфаркт и тд)


In [127]:
# наши таргеты содержатся в report и будут доступны после парсинга, разораться в них поможет суперкласс
Y.report

ecg_id
1                   sinusrhythmus periphere niederspannung
2                      sinusbradykardie sonst normales ekg
3                               sinusrhythmus normales ekg
4                               sinusrhythmus normales ekg
5                               sinusrhythmus normales ekg
                               ...                        
21833    ventrikulÄre extrasystole(n) sinustachykardie ...
21834    sinusrhythmus lagetyp normal qrs(t) abnorm    ...
21835    sinusrhythmus lagetyp normal t abnorm in anter...
21836    supraventrikulÄre extrasystole(n) sinusrhythmu...
21837    sinusrhythmus p-sinistrocardiale lagetyp norma...
Name: report, Length: 21837, dtype: object

In [128]:
# посмотрим, какие варианты у суперкласса
Y['diagnostic_superclass'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[pathology]          11902
[NORM]                9083
[NORM, pathology]      445
[]                     407
Name: diagnostic_superclass, dtype: int64

In [ ]:
data_X = load_raw_data_X(Y, sampling_rate, path)

In [28]:
# уберем пустые значения суперкласса, а также те, которые будут нас путать (нормальные ЭКГ, смешанные с ненормальными)
# оставим два основных для нас информационных столбца: нормальное/не нормальное ЭКГ и патология/нет
norm = []
pathology = []
drop = []
for ind, i in enumerate(Y['diagnostic_superclass']): 
    if len(i)==0:
        drop.append(ind+1)
    elif len(i) > 1 and 'NORM' in i:
        drop.append(ind+1)     
    elif 'NORM' in i:
        norm.append(ind+1)
    elif 'pathology' in i:
        pathology.append(ind+1)
        
Y = Y.drop([d for d in drop])
Y['norm_ecg'] = 0
Y['pathology'] = 0

for ind in Y.index:
    if ind in norm:
        Y['norm_ecg'][ind] = 1 
    elif ind in pathology:
        Y['pathology'][ind] = 1    

/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [29]:
Y

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,diagnostic_superclass,norm_ecg,pathology,ecg_id
0,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr,[NORM],0,0,0
1,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,...,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr,[NORM],1,0,1
2,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,...,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr,[NORM],1,0,2
3,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,...,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr,[NORM],1,0,3
4,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,...,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr,[NORM],1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20980,17180.0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-05-31 09:14:35,ventrikulÄre extrasystole(n) sinustachykardie ...,...,NaN,1ES,NaN,7,records100/21000/21833_lr,records500/21000/21833_hr,[pathology],0,1,20980
20981,20703.0,93.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm ...,...,NaN,NaN,NaN,4,records100/21000/21834_lr,records500/21000/21834_hr,[NORM],0,1,20981
20982,19311.0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-08 10:30:27,sinusrhythmus lagetyp normal t abnorm in anter...,...,NaN,NaN,NaN,2,records100/21000/21835_lr,records500/21000/21835_hr,[pathology],1,0,20982
20983,8873.0,64.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-09 18:21:49,supraventrikulÄre extrasystole(n) sinusrhythmu...,...,NaN,SVES,NaN,8,records100/21000/21836_lr,records500/21000/21836_hr,[NORM],0,1,20983


In [30]:
Y['ecg_id'] = Y.index

In [31]:
Y.index = range(0, Y.shape[0])

In [32]:
Y[['diagnostic_superclass', 'norm_ecg', 'pathology', 'ecg_id']]

,diagnostic_superclass,norm_ecg,pathology,ecg_id
0,[NORM],0,0,0
1,[NORM],1,0,1
2,[NORM],1,0,2
3,[NORM],1,0,3
4,[NORM],1,0,4
...,...,...,...,...
20980,[pathology],0,1,20980
20981,[NORM],0,1,20981
20982,[pathology],1,0,20982
20983,[NORM],0,1,20983


In [33]:
# первая контрольная точка (патология/нормальное ЭКГ)
Y.to_csv('Y.csv')
Y = pd.read_csv('Y.csv')

/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
Y.shape

(20985, 32)

In [35]:
# наш таргет - report - не забыть обработать
Y[Y['norm_ecg']==0].report

0                   sinusrhythmus periphere niederspannung
8                               sinusrhythmus normales ekg
19                              sinusrhythmus normales ekg
22               sinusrhythmus linkstyp sonst normales ekg
24                              sinusrhythmus normales ekg
                               ...                        
20976    ventrikulÄre extrasystole(n) sinusrhythmus lin...
20977    sinusrhythmus lagetyp normal periphere nieders...
20980    ventrikulÄre extrasystole(n) sinustachykardie ...
20981    sinusrhythmus lagetyp normal qrs(t) abnorm    ...
20983    supraventrikulÄre extrasystole(n) sinusrhythmu...
Name: report, Length: 11903, dtype: object

In [36]:
# чтобы определить главные таргеты из report, вынесем все в признаки в отдельный файл
target_values = []
target_ill = []

for ind, target in enumerate(Y[Y['norm_ecg']==0].report):
    for key, value in target_dict.items():
             for v in value:           
                if v in target:
                    target_values.append([key, ind])
                    target_ill.append(target)

In [37]:
# проверить, что все возможные варианты report учтены
for ind, target in enumerate(Y[Y['norm_ecg']==0].report):
    if target not in target_ill:
        for key, value in target_dict.items():
             for v in value:           
                if v not in target:
                    print(target)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp normales ekg
linkstyp norma

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-ekg : normal belastung : normal
ruhe-ekg zur belastung: ruhe-e

In [38]:
# наши целевые таргеты

sinus_rythm = []
arithmia = []
bradycardia = []
tachycardia = []
fibrillation = []
nan_ill = []
questions_ill = []

for target in target_values:
    if target[0] == 'sinus' and target[1] not in sinus_rythm:
        sinus_rythm.append(target[1])
    elif target[0] == 'arithmia' and target[1] not in arithmia:
        arithmia.append(target[1])
    elif target[0] == 'bradykardie' and target[1] not in bradycardia:
        bradycardia.append(target[1])
    elif target[0] == 'tahicardia' and target[1] not in tachycardia:
        tachycardia.append(target[1])
    elif target[0] == 'fibrillation' and target[1] not in fibrillation:
        fibrillation.append(target[1])
    elif target[0] == 'Nans' and target[1] not in nan_ill:
        nan_ill.append(target[1])
    elif target[0] == 'dont_now' and target[1] not in questions_ill:
        questions_ill.append(target[1])

Y['sinus_rythm'] = 'normal'
Y['arithmia'] = 'normal'
Y['bradycardia'] = 'normal'
Y['tachycardia'] = 'normal'
Y['fibrillation'] = 'normal'
Y['nan_ill'] = 'normal'
Y['questions_ill'] = 'normal'

add_col = ['sinus_rythm', 'arithmia', 'bradycardia', 'tachycardia', 'fibrillation', 'nan_ill', 'questions_ill']
add_values = {'sinus_rythm': sinus_rythm, 
              'arithmia': arithmia, 
              'bradycardia' :bradycardia, 
              'tachycardia': tachycardia, 
              'fibrillation': fibrillation,
              'nan_ill': nan_ill, 
              'questions_ill': questions_ill}


In [39]:
for ind in sinus_rythm:
    Y['sinus_rythm'][ind] = 'abnormal'
for ind in arithmia:
    Y['arithmia'][ind] = 'abnormal'
for ind in bradycardia:
    Y['bradycardia'][ind] = 'abnormal'
for ind in tachycardia:
    Y['tachycardia'][ind] = 'abnormal'
for ind in fibrillation:
    Y['fibrillation'][ind] = 'abnormal'
for ind in questions_ill:
    Y['questions_ill'][ind] = 'abnormal'
        

/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

In [31]:
Y[['sinus_rythm', 'arithmia', 'bradycardia', 'tachycardia', 'fibrillation', 'nan_ill', 'questions_ill',
   'norm_ecg', 'report', 'ecg_id']][Y['norm_ecg']==0].groupby('fibrillation').head(2000)

,sinus_rythm,arithmia,bradycardia,tachycardia,fibrillation,nan_ill,questions_ill,norm_ecg,report,ecg_id
7,1,0,0,0,0,0,0,0,sinusrhythmus linkstyp qrs(t) abnormal infe...,8
18,1,0,0,0,0,0,0,0,sinusrhythmus linkstyp unspezifisches abnormal...,22
21,1,0,0,0,0,0,0,0,sinusrhythmus verdacht auf p-sinistrocardiale ...,26
23,1,0,0,0,0,0,0,0,sinusrhythmus verdacht auf p-sinistrocardiale ...,28
25,1,0,0,0,0,0,0,0,sinusrhythmus unspezifischer intraventrikulÄre...,30
...,...,...,...,...,...,...,...,...,...,...
11856,0,0,0,1,1,0,1,0,sinus rhythm. possible left atrial enlargement...,12318
11870,0,0,0,1,1,0,1,0,premature ventricular contraction(s). sinus rh...,12333
11875,0,0,0,0,1,0,1,0,sinusbradykardie llt kammerhypertrophie lvh si...,12338
11880,0,0,0,0,1,0,1,0,atrial fibrillation. st segments are depressed...,12343


In [46]:
ill_df = Y[['sinus_rythm', 'arithmia', 'bradycardia', 'tachycardia', 'fibrillation', 'ecg_id']][Y['norm_ecg']==0]

In [40]:
# ill_df.to_csv('ill_data.csv', index=False)
ill_data = pd.read_csv('ill_data.csv')
ill_data

,sinus_rythm,arithmia,bradycardia,tachycardia,fibrillation,ecg_id
0,1,0,0,0,0,8
1,1,0,0,0,0,22
2,1,0,0,0,0,26
3,1,0,0,0,0,28
4,1,0,0,0,0,30
...,...,...,...,...,...,...
11897,0,0,0,0,0,21827
11898,0,0,0,0,0,21828
11899,0,0,0,0,0,21832
11900,0,0,0,0,0,21833


In [82]:
ill_data['multi_ill'] = 0
ill_data.columns.tolist()[:-2]
dict_ill = {
    'sinus_rythm': 1,
    'arithmia': 2,
    'bradycardia': 3,
    'tachycardia': 4,
    'fibrillation': 5
}

In [96]:
for ill in dict_ill.keys():
    ill_data.loc[(ill_data[ill]==1), 'multi_ill'] = dict_ill[ill]
    

In [95]:
ill_data[ill_data['multi_ill']==0]['fibrillation'].value_counts()

0    5672
Name: fibrillation, dtype: int64

In [97]:
ill_data['multi_ill'].value_counts()

0    5672
1    4230
5     774
2     705
4     349
3     172
Name: multi_ill, dtype: int64

In [58]:
multi_ill

[[0, 1],
 [1, 1],
 [2, 1],
 [3, 1],
 [4, 1],
 [5, 1],
 [6, 1],
 [7, 1],
 [8, 1],
 [9, 1],
 [10, 1],
 [11, 1],
 [12, 1],
 [13, 1],
 [14, 1],
 [15, 1],
 [16, 1],
 [17, 1],
 [18, 1],
 [19, 1],
 [20, 1],
 [21, 1],
 [22, 1],
 [23, 1],
 [24, 1],
 [26, 1],
 [27, 1],
 [28, 1],
 [29, 1],
 [30, 1],
 [31, 1],
 [32, 1],
 [34, 1],
 [35, 1],
 [36, 1],
 [37, 1],
 [38, 1],
 [39, 1],
 [40, 1],
 [41, 1],
 [42, 1],
 [43, 1],
 [45, 1],
 [46, 1],
 [47, 1],
 [50, 1],
 [51, 1],
 [52, 1],
 [53, 1],
 [54, 1],
 [55, 1],
 [56, 1],
 [57, 1],
 [58, 1],
 [59, 1],
 [61, 1],
 [62, 1],
 [63, 1],
 [64, 1],
 [66, 1],
 [67, 1],
 [68, 1],
 [69, 1],
 [70, 1],
 [71, 1],
 [72, 1],
 [73, 1],
 [74, 1],
 [76, 1],
 [77, 1],
 [78, 1],
 [80, 1],
 [81, 1],
 [82, 1],
 [83, 1],
 [85, 1],
 [86, 1],
 [88, 1],
 [90, 1],
 [91, 1],
 [92, 1],
 [94, 1],
 [95, 1],
 [97, 1],
 [100, 1],
 [102, 1],
 [103, 1],
 [105, 1],
 [106, 1],
 [107, 1],
 [108, 1],
 [109, 1],
 [110, 1],
 [112, 1],
 [113, 1],
 [114, 1],
 [116, 1],
 [117, 1],
 [118, 1],
 [119

In [57]:
ill_data['multi_ill'].unique()

array([0])

In [43]:
ill_data['arithmia'].value_counts()

0    10800
1     1102
Name: arithmia, dtype: int64

In [103]:
# ill_data.to_csv('ill_multi.csv', index=False)
ill_df = pd.read_csv('ill_multi.csv')

In [104]:
ill_df

,sinus_rythm,arithmia,bradycardia,tachycardia,fibrillation,ecg_id,multi_ill
0,1,0,0,0,0,8,1
1,1,0,0,0,0,22,1
2,1,0,0,0,0,26,1
3,1,0,0,0,0,28,1
4,1,0,0,0,0,30,1
...,...,...,...,...,...,...,...
11897,0,0,0,0,0,21827,0
11898,0,0,0,0,0,21828,0
11899,0,0,0,0,0,21832,0
11900,0,0,0,0,0,21833,0


In [50]:
# Y.to_csv('Y_current.csv', index=False)


In [99]:
# надо будет удалить экг, где неясно, к чему относится таргет
drop_data = Y[['ecg_id', 'questions_ill']][(Y['norm_ecg']==0)&(Y['questions_ill']==1)]
drop_ecg_id = drop_data['ecg_id'].tolist()
len(drop_ecg_id)

3381

In [101]:
train_signal = pd.read_csv('train_signal.csv')
test_signal = pd.read_csv('test_signal.csv')
valid_signal = pd.read_csv('valid_signal.csv')


In [105]:
# к каждому фрейму присоединить таргеты
train_df = train_signal.join(ill_df.set_index('ecg_id'), on='ecg_id')
test_df = test_signal.join(ill_df.set_index('ecg_id'), on='ecg_id')
valid_df = valid_signal.join(ill_df.set_index('ecg_id'), on='ecg_id')
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
valid_df = valid_df.fillna(0)

In [106]:
# надо удалить экг, где неясно, к чему относится таргет
train_df.drop(pd.Index(np.where(train_df['ecg_id'].isin([i for i in drop_ecg_id]))[0]), inplace = True)
test_df.drop(pd.Index(np.where(test_df['ecg_id'].isin([i for i in drop_ecg_id]))[0]), inplace = True)
valid_df.drop(pd.Index(np.where(valid_df['ecg_id'].isin([i for i in drop_ecg_id]))[0]), inplace = True)


In [116]:
# сохранить локально готовые данные теста, валидации и тренировочные
train_df.to_csv('train_multi.csv', index=False)
train_df = pd.read_csv('train_multi.csv')
train_df.head(10)

,ecg_id,channel-0,channel-1,channel-2,channel-3,channel-4,channel-5,channel-6,channel-7,channel-8,channel-9,channel-10,channel-11,sinus_rythm,arithmia,bradycardia,tachycardia,fibrillation,multi_ill
0,1,-0.119,-0.055,0.064,0.086,-0.091,0.004,-0.069,-0.031,0.000,-0.026,-0.039,-0.079,0.0,0.0,0.0,0.0,0.0,0.0
1,1,-0.116,-0.051,0.065,0.083,-0.090,0.006,-0.064,-0.036,-0.003,-0.031,-0.034,-0.074,0.0,0.0,0.0,0.0,0.0,0.0
2,1,-0.120,-0.044,0.076,0.082,-0.098,0.016,-0.058,-0.034,-0.010,-0.028,-0.029,-0.069,0.0,0.0,0.0,0.0,0.0,0.0
3,1,-0.117,-0.038,0.080,0.077,-0.098,0.021,-0.050,-0.030,-0.015,-0.023,-0.022,-0.064,0.0,0.0,0.0,0.0,0.0,0.0
4,1,-0.103,-0.031,0.072,0.066,-0.087,0.021,-0.045,-0.027,-0.020,-0.019,-0.018,-0.058,0.0,0.0,0.0,0.0,0.0,0.0
5,1,-0.097,-0.025,0.071,0.061,-0.084,0.023,-0.036,-0.025,-0.009,-0.014,-0.012,-0.052,0.0,0.0,0.0,0.0,0.0,0.0
6,1,-0.119,-0.014,0.106,0.066,-0.112,0.046,-0.029,-0.012,0.005,-0.008,-0.007,-0.048,0.0,0.0,0.0,0.0,0.0,0.0
7,1,-0.096,0.008,0.104,0.044,-0.100,0.056,-0.023,0.003,0.018,0.002,-0.001,-0.041,0.0,0.0,0.0,0.0,0.0,0.0
8,1,-0.048,0.044,0.092,0.002,-0.070,0.068,-0.015,0.018,0.021,0.009,0.005,-0.038,0.0,0.0,0.0,0.0,0.0,0.0
9,1,-0.037,0.045,0.081,-0.004,-0.059,0.063,-0.050,0.009,0.018,0.022,0.009,-0.033,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
test_df.to_csv('test_multi.csv', index=False)
valid_df.to_csv('valid_multi.csv', index=False)


test_df